## Extending the capabilities of our model

LLM は非常に有能なツールですが、トレーニングに使用された知識や情報の範囲内でしか機能しません。結局のところ、自分が知っていることしか知らないのです。しかし、トレーニング データに含まれていない質問をする必要がある場合はどうでしょうか。または、トレーニング データには含まれていないが、それに関連する質問をする必要がある場合はどうでしょうか。

この問題を解決するには、所有しているリソースと、それに費やせる時間やお金に応じて、さまざまな方法があります。いくつかのオプションを以下に示します。

- 必要な情報を含めるようにモデルを完全に再トレーニングします。LLM の場合、文字通り数千の GPU を数週間稼働させることができるのは、世界でもほんの一握りの企業だけです。
- この新しい情報を使用してモデルを微調整します。これには必要なリソースが大幅に少なく、通常は数時間または数分で実行できます (モデルのサイズによって異なります)。ただし、モデルを完全に再トレーニングしないため、新しい情報が回答に完全に統合されない可能性があります。微調整は、特定のコンテキストや語彙をよりよく理解することに優れていますが、新しい知識を注入することにはそれほど優れていません。さらに、情報を追加したいときはいつでも、モデルを再トレーニングして再デプロイする必要があります。
- この新しい情報をデータベースに格納し、クエリに関連する部分を取得して、このクエリにコンテキストとして追加してから、LLM に送信します。この手法は、**Retrieval Augmented Generation (RAG、検索拡張生成)** と呼ばれます。この新しい知識を活用するためにモデルを再トレーニングまたは微調整する必要がなく、いつでも簡単に更新できるという点が興味深いです。

[Milvus](https://milvus.io/) を使用してベクター データベースを既に準備しており、そこに [カリフォルニア運転者ハンドブック](https://www.dmv.ca.gov/portal/handbook/california-driver-handbook/) のコンテンツが ([Embeddings](https://www.ibm.com/topics/embedding) の形式で) 保存されています。

このノートブックでは、RAG を使用して **クレームに関するクエリをいくつか実行** し、LLM を変更せずにこの新しい知識がどのように役立つかを確認します。

### 要件とインポート

ラボの指示に従って起動する適切なワークベンチ イメージを選択した場合は、必要なライブラリがすべてすでに用意されているはずです。そうでない場合は、次のセルの最初の行のコメントを解除して、適切なパッケージをすべてインストールします。

In [ ]:
# 適切なワークベンチ イメージを選択していない場合、またはこのノートブックをワークショップ環境外で使用している場合にのみ、次の行のコメントを解除します。
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt

import json

import transformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import VLLMOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from milvus_retriever_with_score_threshold import \
    MilvusRetrieverWithScoreThreshold

# Turn off warnings when downloading the embedding model
transformers.logging.set_verbosity_error()

### Langchain 要素

ここでも、Langchain を使用してタスク パイプラインを定義します。

まず、クエリを送信する **LLM** です。

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://granite-7b-instruct-predictor.ic-shared-llm.svc.cluster.local:8080"

# LLM definition
llm = VLLMOpenAI(           # 私たちは vLLM OpenAI 互換 API クライアントを使用しています。ただし、モデルは OpenAI ではなく OpenShift AI 上で実行されています。
    openai_api_key="EMPTY",   # そのため、これには OpenAI キーは必要ありません。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="granite-7b-instruct",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

次に、カリフォルニア運転者ハンドブックを準備して保存した**ベクター データベース**に接続します。

In [ ]:
# まず、ハンドブックを処理するために使用した埋め込みを定義します
model_kwargs = {"trust_remote_code": True}
embeddings = HuggingFaceEmbeddings(
            model_name="nomic-ai/nomic-embed-text-v1",
            model_kwargs=model_kwargs,
            show_progress=False,
        )

# 次に、Milvusベクトルストアから関連データを取得するリトリーバーを定義します。
retriever = MilvusRetrieverWithScoreThreshold(
            embedding_function=embeddings,
            collection_name="california_driver_handbook_1_0",
            collection_description="",
            collection_properties=None,
            connection_args={
                "host": "vectordb-milvus.ic-shared-milvus.svc.cluster.local",
                "port": "19530",
                "user": "root",
                "password": "Milvus",
            },
            consistency_level="Session",
            search_params=None,
            k=4,
            score_threshold=0.99,
            enable_dynamic_field=True,
            text_field="page_content",
        )

ここで、クエリを作成するために使用する **template** を定義します。このテンプレートには **References** セクションが含まれていることに注意してください。ここに、ベクター データベースから返されたドキュメントが挿入されます。

In [ ]:
template="""<|system|>
You are a helpful, respectful and honest assistant named "Parasol Assistant".
You will be given a claim summary, references to provide you with information, and a question.
You must answer the question based as much as possible on this claim with the help of the references.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.

<|user|>
Claim Summary:
{claim}

References:
{{context}}

Question: {{question}}
<|assistant|>
"""

これで、モデルをクエリする準備ができました!

`claims` フォルダーには、受信できるクレームの例を含む JSON ファイルがあります。最初のクレームを読み取り、それに関連する質問をします。

In [ ]:
# クレームを読み取り、その内容を"claim"変数に格納します

filename = 'claims/claim1.json'

# JSONファイルを開く
with open(filename, 'r') as file:
    data = json.load(file)
claim = data["content"]

### 最初のテスト、追加の知識なし

クレームに関する最初のクエリから始めましょう。ただし、ベクター データベースの助けは借りません。

In [ ]:
# クエリを作成して送信します。

query = "Was Daniel allowed to pass at the red light?"

# 同じテンプレートを異なるタイプのクエリで再利用するためのクイック ハック。
prompt_template = template.format(claim=claim)
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
conversation = prompt | llm

resp = conversation.invoke(input={"context": "", "question": query})

答えが有効であることがわかります。ここで、モデルは交通規制に関する一般的な理解を使用しています。

### 2 回目のテスト、知識を追加

同じプロンプトとクエリを使用しますが、今回はモデルがカリフォルニアのドライバー ハンドブックの参照資料にアクセスします。

In [ ]:
# クエリを作成して送信します。

query = "Was Daniel allowed to pass at the red light?"

prompt_template = template.format(claim=claim)
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
rag_chain = RetrievalQA.from_chain_type(
            llm,
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True,
        )
resp = rag_chain.invoke({"query": query})

非常にすばらしいですね。これで、モデルは遵守すべきルールをより正確に参照するようになりました。

しかし、この情報はどこから得たのでしょうか? ベクター データベースから回答に関連付けられたソースを調べることができます。

In [ ]:
def format_sources(input_list):
    sources = ""
    if len(input_list) != 0:
        sources += input_list[0].metadata["metadata"]["source"] + ', page: ' + str(input_list[0].metadata["metadata"]["page"])
        page_list = [input_list[0].metadata["metadata"]["page"]]
        for item in input_list:
            if item.metadata["metadata"]["page"] not in page_list: # 重複の回避
                page_list.append(item.metadata["metadata"]["page"])
                sources += ', ' + str(item.metadata["metadata"]["page"])
    return sources


results = format_sources(resp['source_documents'])

print(results)

これで完了です。これで、LLM を外部の知識で補完する方法がわかりました。